# Behandling av værdata
<div style="border: 2px solid yellow; padding: 10px; border-radius: 5px; width: 95%; ">
<style>
p {
    line-height: 1.5; 
}
</style>
Her skal vi behandle og manipulere dataen vi har hentet, slik at den blir lettere å lese. Deretter skal vi senere visualisere dette. Vi gjør ulike operasjoner med dataen for å forbedre den for videre analyse, samt forsøker å oppfylle vurderingskriteriene. 

In [1]:
import sys
import json
sys.path.append('../src') 
from data_behandling import DataBehandler
import pandas as pd

# Prøver å lese inn rådatafilen. Dersom filen ikke eksisterer eller er ødelagt,
# håndteres feilen og det opprettes en tom DataFrame for å unngå at programmet krasjer.
try:
    df = pd.read_csv('../data/VaerData.csv')
except FileNotFoundError:
    print("Fant ikke filen 'VaerData.csv'. Har du kjørt HenteData først?")
    df = pd.DataFrame()
except pd.errors.ParserError:
    print("Klarte ikke å lese innholdet i 'VaerData.csv'. Er den kanskje ødelagt?")
    df = pd.DataFrame()

# Åpner og leser Byer.json
byer = DataBehandler.les_byer_fra_fil('../data/Byer.json')

# Oppretter objekt og omstrukturer dataen
behandler = DataBehandler(df, byer)

df = behandler.omstrukturerer_data()

df.head()

# Lagrer den behandlede dataen i csv-filen "BehandletVaerData" slik at vi kan sammenligne rådataen med den vi har behandlet.
behandler.lagre_data()


Behandlet data lagret i: ../data/BehandletVaerData.csv


<div style="border: 2px solid yellow; padding: 10px; border-radius: 5px; width: 95%; ">
<style>
p {
    line-height: 1.5; 
}
</style>
I koden over leses rådata-filen først, deretter behandles og omstruktureres den slik at den blir mer oversiktlig og lettere å lese. 
</div>

In [2]:
import sys
sys.path.append('../src')

from data_behandling import DataBehandler
import pandas as pd

# Leser inn BehandletVaerData.csv som inneholder behandlet værdata.
# Med denne try-except-blokken sikrer vi at notebooken ikke krasjer hvis filen mangler – 
# altså at vi kan håndtere situasjoner der filen ikke finnes eller er skadet.
try:
    df = pd.read_csv('../data/BehandletVaerData.csv')
except FileNotFoundError:
    print("Klarte ikke å finne filen 'BehandletVaerData.csv'. Har du kjørt HenteData først?")
    df = pd.DataFrame()
except pd.errors.ParserError:
    print("Klarte ikke å lese CSV-filen. Er den kanskje skadet?")
    df = pd.DataFrame()

# Oppretter et objekt av klassen DataBehandler slik at vi kan bruke metodene i klassen
behandler = DataBehandler(df, byer)

print(f"Manglende verdier før behandling: \n{behandler.finn_manglende_verdier()}")

# Printer ut radene hvor det mangler verdier
print(f"\nRader med manglende verdier: \n {df[df.isnull().any(axis=1)].index + 2}")

# Behandler manglende verdier
df = behandler.fyll_manglende_codequality()

print(f"\nManglende verdier etter behandling: \n{behandler.finn_manglende_verdier()}")

#Lagrer i BehandletVaerData.csv
behandler.lagre_data()

Manglende verdier før behandling: 
by                0
sourceId          0
referenceTime     0
statistikk        0
variable          0
value             0
unit              0
codequality      10
dtype: int64

Rader med manglende verdier: 
 Index([287, 320, 1281, 1382, 1415, 1431, 1437, 1440, 1626, 2382], dtype='int64')

Manglende verdier etter behandling: 
by               0
sourceId         0
referenceTime    0
statistikk       0
variable         0
value            0
unit             0
codequality      0
dtype: int64

Behandlet data lagret i: ../data/BehandletVaerData.csv


<div style="border: 2px solid yellow; padding: 10px; border-radius: 5px; width: 95%; ">
<style>
p {
    line-height: 1.5; 
}
</style>
Her bruker vi Pandas til å identifisere manglende verdier i rådataen. Vi ser at det kun er i kodekvalitet-kolonnen at det er manglende verdier. Det er også listet opp hvilke rader som har manglende verdier. 

Etter vi analyserte og identifiserte hvilke rader som hadde manglende verdier, fylte vi inn verdier der det opprinnelig manglet. Vi fant ut av hvilken kodekvalitet som var den hyppigste, og fylte inn den der det var manglende verdier. 
</div>

In [3]:
# Klassifiserer "codequality"-verdiene
df = behandler.klassifiser_codequality()

#Lagrer i BehandletVaerData.csv
behandler.lagre_data()


Behandlet data lagret i: ../data/BehandletVaerData.csv


<div style="border: 2px solid yellow; padding: 10px; border-radius: 5px; width: 95%; ">
<style>
p {
    line-height: 1.5; 
}
</style>
I koden over kaller vi på to metoder som ligger i Python-filen "data_behandling", klassifiser_codequality og lagre_data. Disse metodene gjør om kodekvalitetverdiene fra tall og klassifiserer dem i kategoriene "God", "Middels" og "Dårlig", og lagrer den modifiserte og behandlede dataen i csv-filen BehandletVaerData. 
</div>

# Pandas SQL 
<div style="border: 2px solid yellow; padding: 10px; border-radius: 5px; width: 95%; ">
<style>
p {
    line-height: 1.5; 
}
</style>
Ved å benytte oss av Pandas SQL får vi muligheten til å kombinere flere operasjoner. Som for eksempel i spørringen under hvor vi har kunnet hente ut gjennomsnittsverdier for de ulike parameterne for de forskjellige byene. Samtidig som vi kun har hentet den dataen med "God" kodekvalitet. 
</div>


In [4]:
from pandasql import sqldf

query = """
SELECT by as By, variable as Parameter, AVG(value) as Gjennomsnittsverdi
FROM df
WHERE codequality = 'God'
GROUP BY By, Parameter
ORDER BY By DESC
"""
result_df = sqldf(query, locals())
print(result_df)

          By                 Parameter  Gjennomsnittsverdi
0  Trondheim       air_temperature P1D            5.539058
1  Trondheim  precipitation_amount P1D            2.672877
2       Oslo       air_temperature P1D            7.741061
3       Oslo  precipitation_amount P1D            1.926301
4       Oslo            wind_speed P1D            2.626027
5     Bergen       air_temperature P1D            8.924384
6     Bergen  precipitation_amount P1D            6.705479
7     Bergen            wind_speed P1D            3.571507


<div style="border: 2px solid yellow; padding: 10px; border-radius: 5px; width: 95%; ">
<style>
p {
    line-height: 1.5; 
}
</style>

Å benytte oss av SQL gjør jobben med å hente ut valgri data vesentlig lettere synes vi. Koden som kreves for å lykkes, er mye enklere å forstå, samt på et standardisert format som gjør oppsettet enkelt. Her henter vi for eksempel ut hyppigheten av de ulike kodekvalitetene for hver By. Da får vi det svart på hvit at det meste av dataen vi har hentet er kategorisert som "God". Mens det finnes noen få tilfeller for Oslo og Trondheim hvor kodekvaliteten istedenfor er kategorisert som "Dårlig" eller "Middels".
</div>

In [5]:
query = """
SELECT by as By, codequality as Kodekvalitet, COUNT(*) as Antall
FROM df
GROUP BY By, Kodekvalitet
ORDER BY By DESC
"""
result_df = sqldf(query, locals())
print(result_df)

          By Kodekvalitet  Antall
0  Trondheim       Dårlig       4
1  Trondheim          God     726
2       Oslo          God    1088
3       Oslo      Middels       7
4     Bergen          God    1095


In [6]:
sys.path.append('../src') 
from tidsintervall import start_dato, slutt_dato

# Sjekker hvilke dager som har temperaturer under 0 grader
kalde_dager = behandler.tell_kalde_dager()
print(f"Antall kalde dager mellom {start_dato} og {slutt_dato}: {len(kalde_dager)}")

Fant 140 kalde dager.
Antall kalde dager mellom 2022-01-01 og 2023-01-01: 140


<div style="border: 2px solid yellow; padding: 10px; border-radius: 5px; width: 95%; ">
<style>
p{
    line-height:1.5;
}
</style>
For å ha oversikt over antall dager med minusgrader har vi her kalt på metoden tell_kalde_dager for å finne ut av nettopp det. Det kan være nyttig informasjon å ha til videre analyse av værdataen. I funksjonen tell_kalde_dager har vi brukt list comprehensions for å gjøre koden kortere og lettere å forstå. 
</div>

<div style="border: 2px solid yellow; padding:10px; border-radius: 5px; width: 95%;">
<style>
p{
    line-height:1.5;
}
</style>

## Uregelmessigheter i dataene og håndtering av dem
Vi forventer å møte på uregelmessigheter i dataene våre når vi jobber med dem fremover, og vi har også gjort det allerede. Vi har for eksempel erfart at kodekvaliteten var ganske varierende og uregelmessig, og det håndterte vi ved å endre dataen til et mer forståelig format og fylle inn for de manglende verdiene med den vanligste verdien. Det gjorde vi fordi det statistisk sett er mer sannsynlig at den hyppigste verdien skal være der det er manglende verdier enn noen andre. 

I tillegg til manglende verdier er det sannsynlig at vi kan møte på f.eks. feilaktige verdier og feilaktige enheter. Den overordnede planen vår for håndtering av uregelmessigheter er at vi må identifisere problemet, deretter rense og formatere dataene og til slutt loggføre uregelmessighetene. Å loggføre uregelmessighetene kan være lurt for å ha en oversikt over hvilke endringer som er gjort med uregelmessig data. 
</div>